In [ ]:
# import mindspore
# param_dict = mindspore.load_checkpoint("/app/pytorch_GAN_zoo/arbitrary_image_stylization/checkpoint/ais.ckpt")

In [3]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu2(self.conv2(x))
        x = self.maxpool(x)
        
        return x
new_model = SimpleCNN()
parameters = {}


with open('model_parameter.txt', 'r') as f:
    current_section = None
    current_parameters = {}
    for line in f:
        line = line.strip()
        if "optimizer.network" in line or "network.network" in line:
            if current_section is not None:
                parameters[current_section] = current_parameters
            current_section = line
            current_parameters = {}
        else:
            if line:
                key, value = line.split(':')
                current_parameters[key.strip()] = value.strip()
    if current_section is not None:
        parameters[current_section] = current_parameters
checkpoint_params = parameters

for name, param in new_model.named_parameters():
    if name in checkpoint_params:
        param.data = checkpoint_params[name]

torch.save(new_model.state_dict(), 'new_model_checkpoint.pt')


In [4]:
model = SimpleCNN()
ckpt = torch.load('new_model_checkpoint.pt')
model.load_state_dict(ckpt)
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [5]:
print(model.parameters())

<generator object Module.parameters at 0x7f802e9366c0>
